# Apriori Analysis

load all the necessary files 

In [1]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('aisles.csv')
department = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

create a dataset containing order_id, user_id and product_id

In [2]:
order_user_id = orders[['order_id','user_id']]

orderid_productsid_train = order_products_train[['order_id','product_id']]
orderid_productsid_prior = order_products_prior[['order_id','product_id']]
order_products_id = orderid_productsid_train.append(orderid_productsid_prior, ignore_index=True)

all_users_products=order_user_id.merge(order_products_id,left_on='order_id', right_on='order_id')

all_users_products

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


In [3]:
all_users_products['product_id'].max() #49688

all_users_products['user_id'].max() #206209

206209

seperate products in the aisles fresh fruits, vegetables and packaged vegetables and fruits

In [5]:
aisle_ids = []
aisle_array = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
for i in aisle_array:
    aisle_ids.append(i[0])
product_ids = []
product_array = products[products['aisle_id'].isin(aisle_ids)].values
for j in product_array:
    product_ids.append(j[0])
print(len(product_ids))
print(aisle_ids)

1566
[24, 83, 123]


when analyzing the datasets through word cloud found that most of the products purchased belong to the aisles of fruits and vegetable. To understand the user shopping behavior deeper, we should delete the products belonging to these aisles.

remove products of the asiles : fresh fruits(24),fresh vegetables(83) and packaged vegetables fruits(122)

In [6]:
users_products_without_fresh = all_users_products[~all_users_products['product_id'].isin(product_ids)]

In [7]:
users_products_without_fresh

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


here we created 3 functions:

1. to create a dataframe containing user_id and product_id with rows as no.of user_id and columns as no.of order_id.
if a user didnt purchase a particular product then it is saved as false else true

2. to transform association Rules to products

3. transform frequent Item to products

In [8]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

def create_dataframe(user_products):
    user_product_dict={}
    for i in user_products.itertuples():
        if(user_product_dict.get(i[2])):
            user_product_dict[i[2]].append(i[3])
        else:
            user_product_dict[i[2]] = [] 
            user_product_dict[i[2]].append(i[3])  #create dictionary with userid and his/her products 
            #example: {1: [196, 14084, 12427, 26088, 26405, 196, 10258, 12427, 13176, 26088]}
    user_product_list = []
    for j in user_product_dict:
        user_product_list.append(user_product_dict[j]) #create a list of all products purchased by a user
        #example: [196, 14084, 12427, 26088, 26405, 196, 10258, 12427, 13176, 26088]
    encoder = TransactionEncoder() #if product is purchased then entered as True else false
       #example: 196	10258	12427	13176	14084	26088	26405
          #0	True	True	True	True	True	True	True
    encoder_array = encoder.fit(user_product_list).transform(user_product_list)
    df = pd.DataFrame(encoder_array, columns=encoder.columns_) #after encoding converted into dataframe
    return df

In [9]:
def transform_associationRule_to_product(association_rule):
    relationship_rule = association_rule[['antecedents','consequents']]
    array_relation=relationship_rule.values
    final_rules = []
    for item in array_relation:
        product_A_Arr = []
        product_B_Arr = []
        A = item[0]
        B = item[1]
        for pre_item in A:
            product_A_Arr.append(products[products['product_id' ]== pre_item]['product_name'].values[0])
        for after_item in B:
            product_B_Arr.append(products[products['product_id' ]== after_item]['product_name'].values[0])
        tuples = (product_A_Arr,product_B_Arr)
        final_rules.append(tuples)
    return final_rules

In [10]:
def transform_frequentItem_to_product(frequent_itemsets):
    array_relation=frequent_itemsets.values
    final_array = []
    for i in array_relation:
        arr = []
        for j in i[1]:
            arr.append(products[products['product_id' ]== j]['product_name'].values[0])
        final_array.append(arr)
    return final_array

for all_users_products..generate dataframe with only user_id and product_id

generate frequent itemsets for the dataframe using apriori function

generate associate rules for the frequent itemsets using association rules function

convert these rules and frequent items lists into product names

In [12]:
df = create_dataframe(all_users_products) #shape 206209 * 49688
#df[196][0] #True

#we can generate frequent item sets that have a support of at least 1%
frequent_itemsets_all_products = apriori(df, min_support=0.1, use_colnames=True) #returns support and itemsets single and bouble

#The final step is to generate the rules with their corresponding support, confidence and lift
#antecedents(,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
association_rule_all_products = association_rules(frequent_itemsets_all_products, metric="confidence", min_threshold=0.1)

#antecedents and consequents which are product ids are converted into product names
transform_associationRule_to_product(association_rule_all_products)

#listed frequent items
frequent_ascending = frequent_itemsets_all_products.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(frequent_ascending)

[['Banana'],
 ['Bag of Organic Bananas'],
 ['Organic Strawberries'],
 ['Organic Baby Spinach'],
 ['Large Lemon'],
 ['Limes'],
 ['Strawberries'],
 ['Organic Hass Avocado'],
 ['Organic Avocado'],
 ['Organic Blueberries'],
 ['Organic Garlic'],
 ['Organic Yellow Onion'],
 ['Organic Zucchini'],
 ['Organic Raspberries'],
 ['Bag of Organic Bananas', 'Organic Strawberries'],
 ['Yellow Onions'],
 ['Cucumber Kirby'],
 ['Organic Grape Tomatoes'],
 ['Organic Strawberries', 'Organic Baby Spinach'],
 ['Seedless Red Grapes']]

for users_products_without_fresh..generate dataframe with only user_id and product_id

generate frequent itemsets for the dataframe using apriori function

generate associate rules for the frequent itemsets using association rules function

convert these rules and frequent items lists into product names 

In [13]:
df1 = create_dataframe(users_products_without_fresh) #shape 206209 * 49688
#df[196][0] #True

#we can generate frequent item sets that have a support of at least 1%
frequent_itemsets_withoutfruits_products = apriori(df1, min_support=0.1, use_colnames=True) #returns support and itemsets single and bouble

#The final step is to generate the rules with their corresponding support, confidence and lift
#antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
association_rule_withoutfruits_products = association_rules(frequent_itemsets_withoutfruits_products, metric="confidence", min_threshold=0.1)

#antecedents and consequents which are product ids.. are converted into product names
transform_associationRule_to_product(association_rule_withoutfruits_products)

#listed frequent items
frequent_ascending_withoutfruits = frequent_itemsets_withoutfruits_products.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(frequent_ascending_withoutfruits)

[['Organic Cilantro'],
 ['Extra Virgin Olive Oil'],
 ['Organic Italian Parsley Bunch'],
 ['Blueberries'],
 ['Organic Whole Milk']]

include all the above declared functions into one

In [53]:
def apply_apriori_functions(dataset,minSup,minCon):
    df = create_dataframe(dataset)
    t1 = time.time()
    frequent_itemsets0 = apriori(df, min_support=minSup, use_colnames=True)
    t2 = time.time()
    print('time t2 - t1',t2-t1)

    association_rule0 = association_rules(frequent_itemsets0, metric="confidence", min_threshold=minCon)
    # association_rule0
    
    s = frequent_itemsets0.sort_values('support',ascending=False)[:20]
    print('top frequent items')
    frequent_items = transform_frequentItem_to_product(s)
    for item in frequent_items:
        print(item)
    
#     return association_rule0[['antecedents','consequents']]
    return transform_associationRule_to_product(association_rule0)

In [107]:
#user_clusters = pd.read_csv('/Users/akhilakumaripuppala/Desktop/instacart-market-basket-analysis/New_Clusters/AgglometricClusterForMoreOrders.csv')
#user_clusters = pd.read_csv('/Users/akhilakumaripuppala/Desktop/instacart-market-basket-analysis/New_Clusters/ClusterWithKMeanForLessOrders.csv')
#user_clusters = pd.read_csv('/Users/akhilakumaripuppala/Desktop/instacart-market-basket-analysis/New_Clusters/ClusterWithKMeanForMoreOrders.csv')
#user_clusters = pd.read_csv('/Users/akhilakumaripuppala/Desktop/instacart-market-basket-analysis/New_Clusters/MiniBatchClusterWithKMeanForLessOrders.csv')
user_clusters = pd.read_csv('/Users/akhilakumaripuppala/Desktop/instacart-market-basket-analysis/New_Clusters/MiniBatchClusterWithKMeanForMoreOrders.csv')
user_clusters = user_clusters[['user_id','label']]
user0 = user_clusters[user_clusters['label']==0]
user1 = user_clusters[user_clusters['label']==1]
user2 = user_clusters[user_clusters['label']==2]
user3 = user_clusters[user_clusters['label']==3]
user4 = user_clusters[user_clusters['label']==4]

In [111]:
#user0
#user1
#user2
#user3
#user4

,user_id,label
27,140,3
93,413,3
185,812,3
238,1004,3
288,1237,3
...,...,...
50525,205559,3
50571,205747,3
50631,205945,3
50644,206016,3


In [112]:
#association items of clustering 0
user_product_part0 = users_products_without_fresh[users_products_without_fresh['user_id'].isin(user0['user_id'])]
#top rules and top items of train data (70%)
x0=apply_apriori_functions(user_product_part0[:int(len(user_product_part0)*0.7)],0.08,0.1)

#association items of clustering 1
user_product_part1 = users_products_without_fresh[users_products_without_fresh['user_id'].isin(user1['user_id'])]
#top rules and top items of train data (70%)
x1=apply_apriori_functions(user_product_part1[:int(len(user_product_part1)*0.7)],0.13,0.1)

#association items of clustering 2
user_product_part2 = users_products_without_fresh[users_products_without_fresh['user_id'].isin(user2['user_id']) ]
#top rules and top items of train data (70%)
x2=apply_apriori_functions(user_product_part2[:int(len(user_product_part2)*0.7)],0.08,0.1)

#association items of clustering 3
user_product_part3 = users_products_without_fresh[users_products_without_fresh['user_id'].isin(user3['user_id']) ]
#top rules and top items of train data (70%)
x3=apply_apriori_functions(user_product_part3[:int(len(user_product_part2)*0.7)],0.08,0.1)

# #association items of clustering 4
# user_product_part0 = users_products_without_fresh[users_products_without_fresh['user_id'].isin(user4['user_id'])]
# #top rules and top items of train data (70%)
# x4=apply_apriori_functions(user_product_part0[:int(len(user_product_part0)*0.7)],0.08,0.1)


time t2 - t1 4.40592622756958
top frequent items
['Organic Cilantro']
['Extra Virgin Olive Oil']
['Organic Italian Parsley Bunch']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Raspberries']
['Original Hummus']
['Unsalted Butter']
['Half & Half']
['Boneless Skinless Chicken Breasts']
['Organic Black Beans']
['Organic Half & Half']
['Organic Basil']
['Feta Cheese Crumbles']
['Organic Garbanzo Beans']
['Organic Coconut Milk']
['Organic Thyme']
['Organic Unsweetened Almond Milk']
time t2 - t1 0.19355106353759766
top frequent items
['Sparkling Water Grapefruit']
['Lime Sparkling Water']
['Sparkling Lemon Water']
['Extra Virgin Olive Oil']
['Sparkling Water Grapefruit', 'Lime Sparkling Water']
['Sparkling Natural Mineral Water']
['Spring Water']
['Organic Half & Half']
['Organic Italian Parsley Bunch']
['Organic Cilantro']
['Peach Pear Flavored Sparkling Water']
['Original Hummus']
['Half & Half']
['Blueberries']
['Sparkling Water Grapefruit', 'Sparkling 

In [116]:
x3

[(['Organic Diced Tomatoes'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Diced Tomatoes']),
 (['Organic Whole Grain Oatmeal Cereal Baby Food'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Whole Grain Oatmeal Cereal Baby Food']),
 (['Organic Whole Grain Oatmeal Cereal Baby Food'],
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  ['Organic Whole Grain Oatmeal Cereal Baby Food']),
 (['Gluten Free SpongeBob Spinach Littles'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Gluten Free SpongeBob Spinach Littles']),
 (['Gluten Free SpongeBob Spinach Littles'],
  ['Kids Sensible Foods Broccoli Littles']),
 (['Kids Sensible Foods Broccoli Littles'],
  ['Gluten Free SpongeBob Spinach Littles']),
 (['First Prunes'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['First Prunes']),
 (['Peach,  Apricot & Banana Stage 2 Baby Food'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & 

In [117]:
output_rules0 = pd.DataFrame(x0)
output_rules0.columns=['first','second']
output_rules0['label']=0

output_rules1 = pd.DataFrame(x1)
output_rules1.columns=['first','second']
output_rules1['label']=1

output_rules2 = pd.DataFrame(x2)
output_rules2.columns=['first','second']
output_rules2['label']=2

output_rules3 = pd.DataFrame(x3)
output_rules3.columns=['first','second']
output_rules3['label']=3

# output_rules2 = pd.DataFrame(x4)
# output_rules2.columns=['first','second']
# output_rules2['label']=4

output_rules = pd.concat([output_rules0,output_rules1,output_rules2,output_rules3],ignore_index=True)
#output_rules = pd.concat([output_rules0,output_rules1,output_rules2],ignore_index=True)
output_rules

,first,second,label
0,[Bunched Cilantro],[Organic Cilantro],0
1,[Organic Cilantro],[Bunched Cilantro],0
2,[Extra Virgin Olive Oil],[Organic Cilantro],0
3,[Organic Cilantro],[Extra Virgin Olive Oil],0
4,[Extra Virgin Olive Oil],[Organic Italian Parsley Bunch],0
...,...,...,...
1283,"[Broccoli & Apple Stage 2 Baby Food, Spinach P...","[Apple and Carrot Stage 2 Baby Food, Baby Food...",3
1284,[Apple and Carrot Stage 2 Baby Food],[Baby Food Stage 2 Blueberry Pear & Purple Car...,3
1285,[Baby Food Stage 2 Blueberry Pear & Purple Car...,"[Apple and Carrot Stage 2 Baby Food, Broccoli ...",3
1286,[Broccoli & Apple Stage 2 Baby Food],"[Apple and Carrot Stage 2 Baby Food, Baby Food...",3


In [118]:
output_rules.to_csv('sample.csv')

In [52]:
user_clusters.head()

,user_id,label
0,7,2
1,15,1
2,17,1
3,21,1
4,27,1
